In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import tensorflow as tf

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sushilkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sushilkumar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
for intent in intents['intents']:
    #print(intent)
    for pattern in intent['patterns']:
        #print(pattern)
        w = nltk.word_tokenize(pattern)
        #print(w)
        words.extend(w)
        #print(words)
        documents.append((w, intent['tag']))
        #print(documents)
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            #print(classes)
            

In [3]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
#print(words)  #pattern questions

In [4]:
words = sorted(list(set(words)))
print(words)

["'s", ',', 'all', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'construction', 'could', 'day', 'different', 'do', 'eligibility', 'exit', 'extension', 'for', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'home', 'how', 'i', 'information', 'is', 'land', 'later', 'loan', 'looking', 'may', 'me', 'name', 'next', 'nice', 'of', 'offered', 'please', 'provide', 'purchase', 'regarding', 'see', 'support', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'type', 'what', 'you', 'your']


In [5]:
classes = sorted(list(set(classes)))
print(classes)

['Home_Loan', 'Home_construction_loan', 'Home_extension_loan', 'Home_purchase_loan', 'Land_purchase_loan', 'Name', 'eligibility', 'exit', 'goodbye', 'greeting', 'loan_types', 'options', 'thanks', 'types_Home_loan']


In [6]:
print (len(documents), "documents")

75 documents


In [7]:
print (len(classes), "classes", classes)

14 classes ['Home_Loan', 'Home_construction_loan', 'Home_extension_loan', 'Home_purchase_loan', 'Land_purchase_loan', 'Name', 'eligibility', 'exit', 'goodbye', 'greeting', 'loan_types', 'options', 'thanks', 'types_Home_loan']


In [8]:
print (len(words), "unique lemmatized words", words)

62 unique lemmatized words ["'s", ',', 'all', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'construction', 'could', 'day', 'different', 'do', 'eligibility', 'exit', 'extension', 'for', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'home', 'how', 'i', 'information', 'is', 'land', 'later', 'loan', 'looking', 'may', 'me', 'name', 'next', 'nice', 'of', 'offered', 'please', 'provide', 'purchase', 'regarding', 'see', 'support', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'type', 'what', 'you', 'your']


In [9]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [10]:
#print(documents)  Qestions with tags

In [11]:
# initializing training data
training = []
output_empty = [0] * len(classes)
print(output_empty)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
for doc in documents:
    # initializing bag of words
    #print(doc)
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]  #questions
    #print(pattern_words)
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    #print(pattern_words)
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        #print(w)
        bag.append(1) if w in pattern_words else bag.append(0)
        #print(bag)
            # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    #print(output_row)
    training.append([bag, output_row])

In [13]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created successfully")

Training data created successfully


In [14]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [15]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created successfully")

Train on 75 samples
Epoch 1/200
75/75 [==============================] - 1s 14ms/sample - loss: 2.6450 - accuracy: 0.0933
Epoch 2/200
75/75 [==============================] - 0s 388us/sample - loss: 2.5602 - accuracy: 0.1867
Epoch 3/200
75/75 [==============================] - 0s 510us/sample - loss: 2.5226 - accuracy: 0.1867
Epoch 4/200
75/75 [==============================] - 0s 473us/sample - loss: 2.4270 - accuracy: 0.1867
Epoch 5/200
75/75 [==============================] - 0s 462us/sample - loss: 2.3180 - accuracy: 0.2000
Epoch 6/200
75/75 [==============================] - 0s 451us/sample - loss: 2.1439 - accuracy: 0.2933
Epoch 7/200
75/75 [==============================] - 0s 392us/sample - loss: 1.9701 - accuracy: 0.4000
Epoch 8/200
75/75 [==============================] - 0s 518us/sample - loss: 1.8362 - accuracy: 0.4800
Epoch 9/200
75/75 [==============================] - 0s 653us/sample - loss: 1.7683 - accuracy: 0.4533
Epoch 10/200
75/75 [==============================] - 

75/75 [==============================] - 0s 368us/sample - loss: 0.3567 - accuracy: 0.8267
Epoch 80/200
75/75 [==============================] - 0s 416us/sample - loss: 0.3829 - accuracy: 0.8267
Epoch 81/200
75/75 [==============================] - 0s 469us/sample - loss: 0.3622 - accuracy: 0.8000
Epoch 82/200
75/75 [==============================] - 0s 469us/sample - loss: 0.3356 - accuracy: 0.8400
Epoch 83/200
75/75 [==============================] - 0s 606us/sample - loss: 0.3506 - accuracy: 0.8400
Epoch 84/200
75/75 [==============================] - 0s 634us/sample - loss: 0.3920 - accuracy: 0.8400
Epoch 85/200
75/75 [==============================] - 0s 576us/sample - loss: 0.3601 - accuracy: 0.8133
Epoch 86/200
75/75 [==============================] - 0s 570us/sample - loss: 0.4087 - accuracy: 0.7867
Epoch 87/200
75/75 [==============================] - 0s 565us/sample - loss: 0.3435 - accuracy: 0.8000
Epoch 88/200
75/75 [==============================] - 0s 566us/sample - loss:

75/75 [==============================] - 0s 546us/sample - loss: 0.3108 - accuracy: 0.8267
Epoch 157/200
75/75 [==============================] - 0s 569us/sample - loss: 0.3793 - accuracy: 0.8000
Epoch 158/200
75/75 [==============================] - 0s 533us/sample - loss: 0.2658 - accuracy: 0.8400
Epoch 159/200
75/75 [==============================] - 0s 504us/sample - loss: 0.3133 - accuracy: 0.8533
Epoch 160/200
75/75 [==============================] - 0s 572us/sample - loss: 0.2592 - accuracy: 0.8400
Epoch 161/200
75/75 [==============================] - 0s 588us/sample - loss: 0.2814 - accuracy: 0.8800
Epoch 162/200
75/75 [==============================] - 0s 477us/sample - loss: 0.3149 - accuracy: 0.8400
Epoch 163/200
75/75 [==============================] - 0s 585us/sample - loss: 0.3114 - accuracy: 0.8267
Epoch 164/200
75/75 [==============================] - 0s 533us/sample - loss: 0.2616 - accuracy: 0.9067
Epoch 165/200
75/75 [==============================] - 0s 507us/sampl

In [16]:
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['What', 'is', 'home', 'loan', '?'], 'Home_Loan'), (['Regarding', 'home', 'loan'], 'Home_Loan'), (['Home', 'Loan'], 'Home_Loan'), (['loan', 'home']